In [1]:
import os
import sys
import vcfpy
import re
import pandas as pd

In [22]:
#specify path of vcf
vcf_path = '/Users/tristanpwdennis/Software/seqtoolbox/analysis-scripts/vcf/vcf/test.vcf'

#import vcf as reader object
reader = vcfpy.Reader.from_path(vcf_path)

# Build header
header = pd.DataFrame.transpose(pd.DataFrame(['#CHROM', 'POS', 'REF', 'ALT'] + reader.header.samples.names))

# Design matrix in list format
X_list = []  

#convert vcf to tab delimited text, convert to dataframe
for record in reader:
    if not record.is_snv():
        continue
    line = [record.CHROM, record.POS, record.REF]
    line += [alt.value for alt in record.ALT]
    line += [call.data.get('GT') or './.' for call in record.calls]
    X_list.append(line)
    df = pd.DataFrame(X_list)

#stick header on top to make final vcf
vcf=header.append(df)   


In [8]:
#specify mapping (from genotype values to codes)
di = {
    "./.": 0,
    "0/0": 1,
    "0/1": 2,
    "1/0": 3,
    "1/1": 4,
}  

df["pos"] = df[0].map(str) + df[1].map(str) + df[2].map(str) + df[3].map(str)